### Profile serial Python and Elementwise Kernel code

In [ ]:
!nsys profile --sample=none --trace=cuda,nvtx --stats=true python3 serial_python_v1.py 10

In [ ]:
!nsys profile --sample=none --trace=cuda,nvtx --stats=true python3 elementwise_kernel_cupy_v2.py 10

In [ ]:
!nsys profile --sample=none --trace=cuda,nvtx --stats=true python3 elementwise_kernel_cupy_v2.py 10